# Train our Philosophical Document Embedding Model

In [1]:
import torch

import loader
import models
import utility


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

c:\Users\lucas\Desktop\UCSD\256\philosophical_oracle\philosophy_oracle\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
train_label_mapping = {
    "aurelius_meditations.txt": 0,
    "beauvoir_ethicsambiguity.txt": 1,
    "effectivealtruism.txt": 2,
    "effectivealtruism_copy.txt": 2,
    "emerson_selfreliance.txt": 3,
    "kierkegaard_presentage.txt": 4,
    "thoreau_walden.txt": 5,
    "epicurus_lettermenoeceus.txt": 6,
    "epicurus_lettermenoeceus_copy.txt": 6
}
test_label_pt_mapping = {
    "philthis_beauvoir.txt": 1,
    "philthis_effectivealtruism.txt": 2,
    "philthis_emerson.txt": 3,
    "philthis_epicurus.txt": 6,
    "philthis_kierkegaard.txt": 4,
    "philthis_stoicism.txt": 0,
    "philthis_thoreau.txt": 5,
}
test_label_mapping = {
    "gpt_beauvoir.txt": 1,
    "gpt_effectivealtruism.txt": 2,
    "gpt_emerson.txt": 3,
    "gpt_epicureanism.txt": 6,
    "gpt_kierkegaard.txt": 4,
    "gpt_stoicism.txt": 0,
    "gpt_thoreau.txt": 5,
}

In [3]:
loader_params = {
    "num_labels": 7,
    "batch_size": 4,
    "chunk_size": 500,
    "chunk_overlap": 50,
    "balance_multiplier": 1.5
}

train_dataloader = loader.get_dataloader("data/train_data", loader_params, train_label_mapping, balance_data=True, print_info=False)
test_dataloader = loader.get_dataloader("data/test_data", loader_params, test_label_mapping, balance_data=False, print_info=False)

Dataloader from ('data/train_data') created with 110 embeddings in 6.9 seconds.
Dataloader from ('data/test_data') created with 42 embeddings in 1.3 seconds.


In [5]:
model_params = {
    "input_dim": 384,
    "hidden_dim": 128,
    "output_dim": loader_params['num_labels'],
    "dropout": 0.5
}
train_params = {
    "epochs": 10,
    "learning_rate": 1e-3
}



model = models.Classifier(**model_params)
optim = torch.optim.Adam(model.parameters(), lr=train_params['learning_rate'])
model.to(device)

print(f"Epochs  || Train Loss | Train Acc || Test Loss | Test Acc")
for epoch in range(train_params['epochs']):
    train_loss = []
    num_correct, num_total = 0, 0
    for iter, (embeddings, labels) in enumerate(train_dataloader):
        embeddings, labels = embeddings.to(device), labels.to(device)        
        optim.zero_grad()
        logits = model(embeddings)
        
        loss, correct_preds = utility.compute_loss_preds(logits, labels)
        loss.backward()
        optim.step()

        num_correct += correct_preds
        num_total += labels.shape[0]
        train_loss.append(loss.cpu().item())

    eval_loss, eval_acc = utility.evaluate_model(model, test_dataloader, device)
    train_loss = torch.tensor(train_loss)
    print(f"{epoch:<3}/{train_params['epochs']:<3} ||    {train_loss.mean():.3f}   |   {num_correct/num_total:.3f}   ||   {eval_loss:.3f}   |  {eval_acc:.3f}  ")

Epochs  || Train Loss | Train Acc || Test Loss | Test Acc
0  /10  ||    1.909   |   0.284   ||   1.832   |  0.301  
1  /10  ||    1.517   |   0.464   ||   1.470   |  0.446  
2  /10  ||    1.081   |   0.634   ||   1.362   |  0.440  
3  /10  ||    0.810   |   0.736   ||   1.447   |  0.434  
4  /10  ||    0.683   |   0.784   ||   1.319   |  0.470  
5  /10  ||    0.543   |   0.816   ||   1.527   |  0.464  
6  /10  ||    0.470   |   0.857   ||   1.412   |  0.488  
7  /10  ||    0.399   |   0.859   ||   1.373   |  0.518  
8  /10  ||    0.323   |   0.920   ||   1.378   |  0.500  
9  /10  ||    0.277   |   0.923   ||   1.282   |  0.548  
